# Topic Modeling with LDA (Latent Dirichlet Allocation)

![Not Sorted LDA Cluster](https://raw.githubusercontent.com/odysan/TopicModelingNB/master/LDANotSorted.png)

![Sorted LDA Cluster](https://raw.githubusercontent.com/odysan/TopicModelingNB/master/LDASorted.png)

## Background Information

The main ideas behind Latent Dirichlet Allocation is that:
* Every document consists of different topics
* Some documents consist of a larger proportion of one topic compared to another
* Each topic has certain words that are more reflective of that topic

For example, if we had topic "Dog", we could say that the words "bark" and "fetch" would be more reflective of that topic.  On the other hand, if we had topic "Cat", we could say that the words "meow" and "litter" would be more reflective of the topic "Cat".

This project uses LDA to cluster abstracts of the National Science Foundation abstracts corpus, and visualizes a small sample of them.  The documents are available from https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/.

## Python Code

The code in this Jupyter Notebook is available here: https://github.com/odysan/TopicModel

-----------------------------------------------------------------------------------------------------------

## Required Python Modules

We will need to import the following Python modules in order to continue:

In [ ]:
from os import walk # traversing the data
from nltk.corpus import stopwords # removing stopwords
from nltk.stem.porter import PorterStemmer # stemming tokens
from nltk import word_tokenize # tokenizing each sentence
import string # removing punctuation
from gensim import corpora, models # for LDA
import numpy as np # for visualization
import matplotlib.pyplot as plt # for visualization
import multiprocessing as mp # for multithreading file reads
from collections import defaultdict # for organizing topic distributions into buckets
import optparse # for command line arguments

## Processing A Research Paper from the National Science Foundation

First, download the archives from https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/.  At the time of writing, part 1, part 2, and part 3 were available.

Scanning a few documents shows that they have document metadata at the top of the file, while the bottom is reserved for the actual abstract -- where it is explicitly delimited "Abstract :".  As well, each abstract is limitted to a certain number of words per line so that the actual abstract is separated into several lines.  We can easily parse through the file by looking for the line that matches the "Abstract :" delimiter.

When it comes to Natural Language Processing, we need to clean the sentence that we will be performing analysis on.  This involves removing punctuation, tokenizing, removing stop words, and stemming each word.

If we come across a character that is not parseable, or an error is raised while parsing, we ignore that abstract altogether.

In [ ]:
def process_file(filepath):
    """Reads the NSF paper and returns the abstract.  Each abstract is tokenized, stripped of its stop words, and
    stemmed.

    Returns string
    """
    found_abstract = False
    processed_line = []

    with open(filepath) as f:
        for line in f:
            if "Abstract    :" in line:
                found_abstract = True
                continue
            if found_abstract:
                try:
                    line = line.lower().translate(None, string.punctuation)
                    splitted_line = word_tokenize(line)
                    removed_stop_words = [word for word in splitted_line if word not in stopwords.words('english')]
                    stemmed_words = [stemmer.stem(i) for i in removed_stop_words]
                    processed_line.extend(stemmed_words)
                except:
                    return -1
    return processed_line

The following function is the callback for the above function.  It is used to analyze the output of the multithreaded child process.  We discard any erroneous data, and ignore any abstract that contains less than 10 words.

In [ ]:
def process_file_callback(return_val):
    """Handles the return value of each child process.  Ignores all abstracts that failed parsing and do not contain
    more than 10 cleaned tokens.
    """
    if return_val != -1 and len(return_val) > 10:
        all_sentences.append(return_val)

## LDA On The Abstracts

Since the gensim library takes care of the LDA algorithm, we won't define a new function just for LDA.  So from this point on, we already ran the LDA algorithm on all of the valid (e.g. non-erroneous and len(abstract) > 10)) abstracts.

## Organizing Documents By Topic (for Visualization)

Visualization is a big part of analytical roles.  We want to be able to see how effectively the algorithm can cluster several different datasets.  Because of this, we need to organize a small sample of our documents so that we can easily see which topic they belong to.

In order to do that, we need to organize them by buckets (which is based on each topic).  Numerical order does not matter so much in this case, since each topic is essentially a categorical value.  We will do this by building a dictionary of topic id:list of documents that mostly reflect that topic id.  ie. documents that talk about dogs will hash to topic "dog", documents that talk about cats will has to topic about "cat".

The first step is to find the dominant topic in a given document.  We then direct that document to its corresponding dominant topic in the dictionary as mentioned above.

We continue to do this until all topics have greater than a pre-specified amount of documents.

In [ ]:
def hash_topics_to_buckets(corp, lda_model, num_of_topics, min_docs):
    """Takes a non-random sample of all of the NSF papers and returns them for the sake of visualization.  Stops when
    there are at least min_docs (int) documents in each topic.

    Returns a dictionary of (k, v) = (topic id, tuple(document id, document's topic distribution))"""
    topics_hash = defaultdict(list)
    vis_topics = []
    arbitrary_id = 0
    for doc_vec in corp:
        max_topic_id = -9999
        max_topic_dist = -9999
        topic_probs = [0] * num_of_topics
        for (topic_id, topic_dist) in lda_model.get_document_topics(doc_vec, minimum_probability=0.005):
            if topic_dist > max_topic_dist:
                max_topic_dist = topic_dist
                max_topic_id = topic_id
            topic_probs[topic_id] = topic_dist
        arbitrary_id += 1
        topics_hash[max_topic_id].append((arbitrary_id, topic_probs))
        vis_topics.append((arbitrary_id, topic_probs))
        finished = True
        for x in range(0, num_of_topics):
            if len(topics_hash[x]) < min_docs:
                finished = False
        if finished:
            # (ids, distributions_matrix) = create_vis_data(vis_topics, num_of_topics)
            # visualize(ids, distributions_matrix)
            break
    return topics_hash

## Creating Visualization-friendly Data

The matplotlib documentation uses numpy arrays a lot, so we will be inserting our data from the dictionary to one numpy array.  Our function will also work with lists, but it won't be organized like how our topic buckets are organized.

The function cycles through each key (which is sorted) and inserts the distribution data into a matrix.  As well, it inserts the document ID into another list for the sake of visualization.

In [ ]:
def create_vis_data(collection, num_of_topics):
    """Based on a dictionary or a list, creates a distribution matrix (using numpy) and list of id's associated with
    each record for the sake of visualization.

    Returns tuple(list of id's, dist matrix)"""
    list_of_ids = []
    dist_matrix = np.vstack([[0] * num_of_topics])
    if isinstance(collection, defaultdict):
        for key in sorted(collection):
            for value in collection[key]:
                dist_matrix = np.vstack([dist_matrix, value[1]])
                list_of_ids.append(value[0])
    elif isinstance(collection, list):
        for tup in collection:
            dist_matrix = np.vstack([dist_matrix, tup[1]])
            list_of_ids.append(tup[0])
    dist_matrix = np.delete(dist_matrix, (0), axis=0)
    return list_of_ids, dist_matrix

## Visualization

Now, with the list of document IDs and the distributions matrix, we can visualize the data.  It sets the document IDs as the x-tick labels, and the topic ID as the y-tick labels.

In [ ]:
def visualize(ids, dists):
    """Visualizes distributions matrix."""
    weights_graph(dists)
    (num_cols, num_rows) = dists.shape
    plt.yticks(range(0, num_rows), range(0, num_rows))
    plt.ylabel('Topic ID')
    plt.xticks(range(0, len(ids)), ids)
    plt.xlabel('Document ID')
    plt.title('Document distributions per topic')
    plt.show()

## Graphing

The data is visualized in a graph.  Each row is a ropic, and each column is a document.  Darker squares mean that the document represents that topic very well.  Lighter squares mean the opposite.  This code is heavily based on the hinton diagram as seen on the matplotlib documentation via http://matplotlib.org/examples/specialty_plots/hinton_demo.html

In [ ]:
def weights_graph(matrix, max_weight=None, ax=None):
    """Draw diagram for visualizing a weight matrix.  Y-axis represents topic ID, X-axis represents document ID.
    Darker squares imply that document represents that topic more than it represents other topics.  The lighter the
    square, the lower its representation of that topic.
    ie. each row is a topic, each column is a document, and darker squares mean that the document represents that
    topic very well.

    Heavily based on the hinton diagram here: http://matplotlib.org/examples/specialty_plots/hinton_demo.html"""
    ax = ax if ax is not None else plt.gca()

    ax.patch.set_facecolor('lightgray')
    ax.set_aspect('equal', 'box')
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    for (x, y), w in np.ndenumerate(matrix):
        color = (0,0,0, w)
        size = 0.9
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size,
                             facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()

## Putting It All Together

By accepting command-line arguments, we can change the data directory that we contain the folders in.  As well, we can change the number of topics that we want to cluster for, specify a different number of iterations of LDA, change the number of workers we use when multithreading processes, and choose a different minimum number of documents per topic when visualizing.

To start, we multithread the function which reads files.  By using an SSD it cuts processing time down to a fraction of what it used to, since we are no longer held back by seek time.

After all abstracts have been read and processed, we prepare a dictionary of all of the different tokens.

We then remove all of the extreme tokens.  Each token that appears in less than 5 documents is filtered out.  Each token that appears in more than half of the documents is filtered out.  Then we create a bag-of-words list for each document.

Next, we run the multithreaded LDA algorithm on the processed abstracts.

Finally, we visualize the results.  The results can be seen from above.

In [ ]:
# Accept command-line arguments
optparser = optparse.OptionParser()
optparser.add_option("-d", "--data", dest='data', default="./data", type=str,
                     help="The directory containing the NSF papers that we will be clustering. (default: ./)")
optparser.add_option("-t", "--numtopics", dest='num_of_topics', default=5, type=int,
                     help="Number of topics we want to group our papers into. (default: 5)")
optparser.add_option("-i", "--iterlda", dest='iterations_lda', default=20, type=int,
                     help="Number of times the LDA algorithm should iterate for topic modeling. (default: 20")
optparser.add_option("-w", "--numworkers", dest='num_of_workers', default=mp.cpu_count(), type=int,
                     help="Number of workers for multithreading (default: max available for CPU)")
optparser.add_option("-k", "--mindocs", dest='min_docs', default=5, type=int,
                     help="Visualize a sample of all documents that reflect at least k of each topic. "
                          "(default: 5)")
(opts, _) = optparser.parse_args()

# Multithreaded read files
pool = mp.Pool(opts.num_of_workers)
stemmer = PorterStemmer()
all_sentences = []
all_words = set()
for (dirpath, dirnames, filenames) in walk(opts.data):
    if len(dirnames) == 0:
        for filename in filenames:
            if filename not in ["links.html", "index.html"]:
                pool.apply_async(process_file, args=(dirpath + "/" + filename, ), callback=process_file_callback)
pool.close()
pool.join()
print "Finished reading file."

# Prepare dictionary of all of the different tokens
dictionary = corpora.Dictionary(all_sentences)
print "Finished preparing dictionary."

# Remove noisy tokens and create bag-of-words list
dictionary.filter_extremes()
corpus = [dictionary.doc2bow(sentence) for sentence in all_sentences]
print "Finished preparing corpus."

# Run multithreaded LDA on processed abstracts
ldamodel = models.LdaMulticore(corpus=corpus, num_topics=opts.num_of_topics, id2word=dictionary,
                               passes=opts.iterations_lda, workers=opts.num_of_workers)
print "Finished building LDA model."

# Prepare buckets for visualization
topics_hash = hash_topics_to_buckets(corpus, ldamodel, opts.num_of_topics, opts.min_docs)
print "Finished sorting by topic."

# Visualize!
(ids, distributions_matrix) = create_vis_data(topics_hash, opts.num_of_topics)
visualize(ids, distributions_matrix)